# Spark Context 설정

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('restaurant-review-average')
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 04:05:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# csv파일을 RDD 로딩

In [6]:
filepath = '/home/ubuntu/working/spark-examples/data/restaurant_reviews.csv'

In [7]:
# 경로가 엉망 진창이더라도 RDD가 만들어지긴함
# Action을 실제로 해야 알 수 있음.

lines = sc.textFile(f'file:///{filepath}') 
lines

file:////home/ubuntu/working/spark-examples/data/restaurant_reviews.csv MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [8]:
# collect() : rdd 내의 데이터를 모두 출력
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [12]:
# header 제거
header = lines.first()

# RDD는 제거, 수정 이런게 없다. 그래서 불러올 때 ~를 제외하고 불러오는 방식으로 제거
datas = lines.filter(lambda row : row != header)

In [14]:
datas.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [23]:
# parsing(HTML-beatifulsoup에서 함)
# lambda로 짜는건 빡세니, def 사용

def parse_task(row):
    # 2번, 3번 필드 반환. 단, 3번 필드는 정수로 리턴
    fields = row.split(',')
    
    category = fields[2]
    review_cnt = int(fields[3])
    
    return category, review_cnt

In [24]:
parse_task('0,짜장면,중식,125,')

('중식', 125)

In [27]:
# 내가 원하는 task 수행 => map

RDD 내의 모든 데이터에 대해 `parse_task` 함수를 적용 후 `map` 으로 추출

In [28]:
category_reviews = datas.map(parse_task)
category_reviews

PythonRDD[10] at RDD at PythonRDD.scala:53

In [30]:
# key_value RDD가 만들어짐
# 비슷 or 똑같은 key를 가진 데이터들을 비슷한, 가까운 partition에 배치가 됨
# 아직 정렬은 아니고, 비슷한 것들끼리 모은 것임.

category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

**카테고리별 리뷰 개수들의 평균**
- value 바꿀 때 웬만하면 map말고 mapValues를 써라.
- 왜냐? 키는 바뀔 일이 없기 때문.
- map을 쓰면 partition이 바뀔 수 있엄.

In [38]:
category_reviews_count = category_reviews.mapValues(lambda x : (x,1))
category_reviews_count.collect()

# def aa(x):
#     return (x, 1)

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

- reduceByKey를 이용해 `Key`별 `Value` 에 대한 집계를 수행
- 집계 Task(함수) 정의가 필요함.

In [40]:
# x= 누적값, y= 새로운값.
reduced = category_reviews_count.reduceByKey(lambda x,y: (x[0] + y[0], x[1]+y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

예를 들어 **중식**의 `x`, `y`
- 첫번째 중식: (125,1) -> x
- 두번째 중식: (235,1) -> y

- 리뷰의 개수끼리 합치기: x[0] + y[0]
- 건수끼리 합치기 : x[1] + y[1]

In [43]:
# 리뷰 개수의 평균 구하기

avg = reduced.mapValues(lambda x: x[0]/x[1])
avg.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [44]:
sc.stop()